In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

## Retrieves the top 10 most viewed videos

In [ ]:
https://www.youtube.com/c/ufc/videos
https://www.youtube.com/c/swissbeatbox
https://www.youtube.com/c/swissbeatbox/videos?view=0&sort=p&flow=grid
https://www.youtube.com/c/swissbeatbox/videos?view=0&sort=da&flow=grid #old
https://www.youtube.com/c/swissbeatbox/videos?view=0&sort=dd&flow=grid #new

https://www.youtube.com/channel/UCLC2zv6DBJpkBIY6Z4tArRQ
https://www.youtube.com/channel/UCLC2zv6DBJpkBIY6Z4tArRQ/videos?view=0&sort=p&flow=grid


https://www.youtube.com/channel/UCktiIeImOu4QzS6fM8gQKPA/videos?view=0&sort=p&flow=grid

def main(type):
    driver = webdriver.Chrome()
    link = input("Paste a youtube link here")
    driver.get('https://www.youtube.com/channel/UCktiIeImOu4QzS6fM8gQKPA/videos?view=0&sort=p&flow=grid')
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content,'lxml')
    #print (soup.prettify())
    titles = soup.findAll('a',id='video-title')
    #print (titles)
    for title in titles:
         print(title.text)

main()

In [3]:
word = input("type a word")
print(word)